In [54]:
import requests
from bs4 import BeautifulSoup
import math
import pandas as pd
from urllib.parse import quote
import re
import time

In [5]:
base_url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?'

In [ ]:
start = 0
keywords = input('Search for : ')
location = input('Location : ')
jobs_scraped = 0

target_url = base_url+f'keywords={quote(keywords)}&location={quote(location)}'

l=[]
o={}
k=[]

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}

print('Job IDs scraped : ')

while start != -1:
    
    res = requests.get(target_url+f'&start={start}')

    if len(re.sub('[\n <!->]','',res.text))>0:

        soup=BeautifulSoup(res.text,'html.parser')
        alljobs_on_this_page=soup.find_all("li")
        
        for x in range(0,len(alljobs_on_this_page)):
            try : 
                jobid = alljobs_on_this_page[x].find("div",{"class":"base-card"}).get('data-entity-urn').split(":")[3]
                l.append(jobid)
            except :
                pass
    
        print(f'{len(l)}')
        start += 25
    
    else :
        start = -1

In [41]:
m = l

In [43]:
l = l[:20]

In [55]:
job_posting_url='https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'

for j in range(0,len(l)):

    resp = requests.get(job_posting_url.format(l[j]))
    soup=BeautifulSoup(resp.text,'html.parser')

    try:
        o["company"]=soup.find("div",{"class":"top-card-layout__card"}).find("a").find("img").get('alt')
    except:
        o["company"]=None

    try:
        o["location"]=soup.find("div",{"class":"topcard__flavor-row"}).find("span",{'class':'topcard__flavor--bullet'}).text.strip()
    except:
        o["location"]=None
    
    try:
        o['posted_time_ago']=soup.find_all("div",{"class":"topcard__flavor-row"})[1].find('span',{'class':'posted-time-ago__text'}).text.strip()
    except:
        o['posted_time_ago']=None
    
    try:
        o['applicant_number']=soup.find_all("div",{"class":"topcard__flavor-row"})[1].find('span',{'class':'num-applicants__caption'}).text.strip()
    except:
        o['applicant_number']=None

    try:
        o["job_title"]=soup.find("div",{"class":"top-card-layout__entity-info"}).find("a").text.strip()
    except:
        o["job_title"]=None

    try :
        o["job_description"]=soup.find("div",{"class":"description__text description__text--rich"}).text.strip()
    except:
        o["job_description"]=None

    try:
        o["level"]=soup.find("ul",{"class":"description__job-criteria-list"}).find("li").text.replace("Seniority level","").strip()
    except:
        o["level"]=None
    try:
        o["employment_type"]=soup.find("ul",{"class":"description__job-criteria-list"}).find_all("li")[1].text.replace("Employment type","").strip()
    except:
        o["employment_type"]=None
    try:
        o["job_function"]=soup.find("ul",{"class":"description__job-criteria-list"}).find_all("li")[2].text.replace("Job function","").strip()
    except:
        o["job_function"]=None
    try:
        o["industries"]=soup.find("ul",{"class":"description__job-criteria-list"}).find_all("li")[3].text.replace("Industries","").strip()
    except:
        o["industries"]=None

    k.append(o)
    o={}

df = pd.DataFrame(k)
timestamp = time.time()
df.to_csv(f'./expo/{timestamp}linkedinjobs.csv', index=False, encoding='utf-8')
print(len(df))

OSError: Cannot save file into a non-existent directory: 'expo'

In [46]:
df

,company,location,posted_time_ago,applicant_number,job_title,job_description,level,employment_type,job_function,industries
0,Talent Disruptors,"Athens, Attiki, Greece",8 hours ago,26 applicants,Data Analyst,None,Entry level,Full-time,Information Technology,Staffing and Recruiting
1,MANDYNAMIC,Greece,5 days ago,None,Data Analyst,None,Entry level,Full-time,Analyst,"Technology, Information and Internet"
2,PwC Greece,"Athens, Attiki, Greece",6 months ago,None,Data Analyst,None,Mid-Senior level,Full-time,Analyst,"Technology, Information and Internet"
3,SILVARE Consulting L.P.,"Athens, Attiki, Greece",2 months ago,197 applicants,Data Analyst,None,Mid-Senior level,Full-time,Information Technology,"Technology, Information and Internet"
4,Unilever,"Athens, Attiki, Greece",2 days ago,None,Data Analyst UFS,None,Entry level,Full-time,Information Technology,"Food and Beverage Services, Food and Beverage ..."
5,Energy Aspects,"Athens, Attiki, Greece",1 week ago,None,Data Analyst - OilX,None,Entry level,Full-time,Information Technology,Oil and Gas
6,VesselBot-Enabling Sustainable Supply Chains,"Athens, Attiki, Greece",1 week ago,40 applicants,Data Analyst,None,Entry level,Full-time,Information Technology,"Technology, Information and Internet"
7,Etraveli Group,"Athens, Attiki, Greece",2 weeks ago,None,Fintech Data Analyst,None,Entry level,Full-time,Analyst,"Technology, Information and Internet"
8,Overmind,"Athens, Attiki, Greece",2 months ago,158 applicants,Data Analyst,None,Entry level,Full-time,Information Technology,"Technology, Information and Internet"
9,Alma Economics,"Athens, Attiki, Greece",3 days ago,124 applicants,Data Analyst,None,Entry level,Full-time,Information Technology and Consulting,Business Consulting and Services


In [52]:
resp=requests.get('https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3821317085')

In [53]:
soup=BeautifulSoup(resp.text,'html.parser')
soup.find("div",{"class":"description__text description__text--rich"}).text.strip()

'We are on the lookout for a Data Analyst with a specialization in marketing analytics. In this critical role, you will harness the power of tools like Google Data Studio, BigQuery, and Power BI to derive actionable insights that drive our marketing decisions. Your expertise in analyzing complex datasets and turning them into understandable and strategic information will be essential in shaping our marketing strategies. This position is ideal for someone who thrives on data-driven challenges and has a passion for leveraging technology to inform business decisions.What You’ll Be DoingData Analysis and Interpretation: Utilize tools like Google Data Studio, BigQuery, and Power BI to analyze marketing data and extract meaningful insights. Developing Reporting Dashboards: Create dynamic dashboards and reports that provide real-time insights into marketing performance. Collaborating with Marketing Teams: Work closely with marketing teams to understand their data needs and provide insights th